In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#carregar o arquivo input_data.py
import input_data
mnist = input_data.read_data_sets('./mnist/', one_hot = True)

In [ ]:
tf.reset_default_graph()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(mnist.train.images[0].reshape(28,28), cmap = 'Greys')

In [ ]:
mnist.train.images[0]

In [ ]:
import numpy as np
imagem1 = np.arange(0,784).reshape(28,28)
plt.imshow(imagem1)

In [ ]:
imagem2 = np.random.normal(size = 784).reshape(28,28)
plt.imshow(imagem2)

In [ ]:
ruido_ph = tf.placeholder(tf.float32, [None, 100])

In [ ]:
def gerador(ruido, reuse = None):
    with tf.variable_scope('gerador', reuse = reuse):
        # 100 -> 128 -> 128 -> 784
        camada_oculta1 = tf.nn.relu(tf.layers.dense(inputs = ruido, units = 128))
        camada_oculta2 = tf.nn.relu(tf.layers.dense(inputs = camada_oculta1, units = 128))
        camada_saida = tf.layers.dense(inputs = camada_oculta2, units = 784, activation = tf.nn.tanh)
        return camada_saida

In [ ]:
imagens_reais_ph = tf.placeholder(tf.float32, [None, 784])

In [ ]:
def discriminador(X, reuse = None):
    with tf.variable_scope('discriminador', reuse = reuse):
        # 784 -> 128 -> 128 -> 1
        camada_oculta1 = tf.nn.relu(tf.layers.dense(inputs = X, units = 128))
        camada_oculta2 = tf.nn.relu(tf.layers.dense(inputs = camada_oculta1, units = 128))
        logits = tf.layers.dense(camada_oculta2, units = 1)
        return logits

In [ ]:
logits_imagens_reais = discriminador(imagens_reais_ph)
logits_imagens_ruido = discriminador(gerador(ruido_ph), reuse = True)

erro_discriminador_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_reais,
                                                                                labels = tf.ones_like(logits_imagens_reais) * (0.9)))
erro_discriminador_ruido = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_ruido,
                                                                                 labels = tf.zeros_like(logits_imagens_ruido)))
erro_discriminador = erro_discriminador_real + erro_discriminador_ruido

erro_gerador = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_ruido,
                                                                     labels = tf.ones_like(logits_imagens_ruido)))


In [ ]:
variaveis = tf.trainable_variables()
variaveis

In [ ]:
variaveis_discriminador = [v for v in variaveis if 'discriminador' in v.name]
print([v.name for v in variaveis_discriminador])

In [ ]:
variaveis_gerador = [v for v in variaveis if 'gerador' in v.name]
print([v.name for v in variaveis_gerador])

In [ ]:
treinamento_discriminador = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(erro_discriminador,
                                                                                  var_list = variaveis_discriminador)
treinamento_gerador = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(erro_gerador,
                                                                            var_list = variaveis_gerador)

In [ ]:
batch_size = 100
amostras_teste = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #ruido_teste = np.random.uniform(-1, 1, size = (1,100))
    #amostra = sess.run(gerador(ruido_ph, True), feed_dict = {ruido_ph: ruido_teste})
    
    #batch = mnist.train.next_batch(100)
    #imagens_batch = batch[0].reshape((100, 784))
    #imagens_batch = imagens_batch * 2 - 1
    #r = sess.run(discriminador(imagens_reais_ph, True), feed_dict = {imagens_reais_ph: imagens_batch})
    #r2 = sess.run(tf.nn.sigmoid(r))
    #ex = tf.constant([[1,2],[3,4]])
    #print(sess.run(tf.ones_like(ex)))
    
    for epoca in range(300):
        numero_batches = mnist.train.num_examples // batch_size
        for i in range(numero_batches):
            batch = mnist.train.next_batch(batch_size)
            imagens_batch = batch[0].reshape((100, 784))
            imagens_batch = imagens_batch * 2 - 1
            
            batch_ruido = np.random.uniform(-1, 1, size = (batch_size, 100))
            
            _, custod = sess.run([treinamento_discriminador, erro_discriminador], 
                                 feed_dict = {imagens_reais_ph: imagens_batch, ruido_ph: batch_ruido})
            _, custog = sess.run([treinamento_gerador, erro_gerador], feed_dict = {ruido_ph: batch_ruido})
            
        print('época: ' + str(epoca + 1) + ' erro D: ' + str(custod) + ' erro G: ' + str(custog))
        
        ruido_teste = np.random.uniform(-1, 1, size = (1, 100))
        imagem_gerada = sess.run(gerador(ruido_ph, reuse = True), feed_dict = {ruido_ph: ruido_teste})
        amostras_teste.append(imagem_gerada)
            
            

In [ ]:
amostra.shape

In [ ]:
plt.imshow(amostra.reshape(28,28))

In [ ]:
batch = mnist.train.next_batch(100)
batch[0].shape
imagens_batch = batch[0].reshape((100, 784))

In [ ]:
imagens_batch.shape

In [ ]:
imagens_batch[0]

In [ ]:
imagens_batch = imagens_batch * 2 - 1
imagens_batch[0]

In [ ]:
r.shape

In [ ]:
r

In [ ]:
r2

In [ ]:
plt.imshow(amostras_teste[299].reshape(28,28))